brief- reading fits and csv files,removing rows with 0,nan and -9999, combining all files based on three catagories,creating dummy variables for flags.making raw data for further project

In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# extracting mass and metallicity from (.fits) file

In [2]:
file1=fits.open('galSpecExtra-dr8.fits')

In [3]:
file1.info()

Filename: galSpecExtra-dr8.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    107   1843200R x 45C   [19A, J, J, J, I, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E]   


In [4]:
file1[0].header

SIMPLE  =                    T /Dummy Created by MWRFITS v1.8                   
BITPIX  =                    8 /Dummy primary header created by MWRFITS         
NAXIS   =                    0 /No data is associated with this header          
EXTEND  =                    T /Extensions may (will!) be present               

In [5]:
file1[1].header

XTENSION= 'BINTABLE'           /Binary table written by MWRFITS v1.8            
BITPIX  =                    8 /Required value                                  
NAXIS   =                    2 /Required value                                  
NAXIS1  =                  193 /Number of bytes per row                         
NAXIS2  =              1843200 /Number of rows                                  
PCOUNT  =                    0 /Normally 0 (no varying arrays)                  
GCOUNT  =                    1 /Required value                                  
TFIELDS =                   45 /Number of columns in table                      
COMMENT                                                                         
COMMENT  *** End of mandatory fields ***                                        
COMMENT                                                                         
COMMENT                                                                         
COMMENT  *** Column names **

In [6]:
data=file1[1].data

In [7]:
data1={'plate':data['PLATEID'],'mjd':data['MJD'],'fiberid':data['FIBERID'],'OH_P50':data['OH_P50'],'LGM_TOT_P50':data['LGM_TOT_P50']}
df=pd.DataFrame(data1)

In [8]:
df.head(100)

,plate,mjd,fiberid,OH_P50,LGM_TOT_P50
0,266,51602,1,-9999.0,10.294701
1,266,51602,2,-9999.0,11.162018
2,266,51602,3,-9999.0,-9999.000000
3,266,51602,4,-9999.0,11.373624
4,266,51602,5,-9999.0,-9999.000000
...,...,...,...,...,...
95,266,51602,96,-9999.0,7.850888
96,266,51602,97,-9999.0,11.439088
97,266,51602,98,-9999.0,10.853505
98,266,51602,99,-9999.0,-9999.000000


In [9]:
df = df.replace(-9999,pd.np.nan)
df=df.replace(0,pd.np.nan)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17200\1460290643.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df = df.replace(-9999,pd.np.nan)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17200\1460290643.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df=df.replace(0,pd.np.nan)


In [10]:

df.to_csv('raw_metallicity_stellarmass.csv',index=False)

In [11]:
f1=pd.read_csv("raw_metallicity_stellarmass.csv")

In [12]:
f1=f1.dropna()

file of mass and metallicity (cleaned data)

In [13]:
f1.to_csv("mass_metallicity.csv",index=False)

In [14]:
m_m=pd.read_csv("mass_metallicity.csv")

In [15]:
m_m.head(10)

,plate,mjd,fiberid,OH_P50,LGM_TOT_P50
0,266,51602,12,9.012159,10.139853
1,266,51602,15,8.932603,9.767632
2,266,51602,16,8.771097,9.364033
3,266,51602,49,9.118535,10.678596
4,266,51602,54,9.050081,10.870289
5,266,51602,60,9.110540,10.519801
6,266,51602,72,8.320573,8.462000
7,266,51602,81,8.951598,9.703085
8,266,51602,85,8.698087,9.268700
9,266,51602,86,8.205136,7.612726


In [16]:
len(m_m)

206980

five bands magnitude file

In [17]:
bands=pd.read_csv("bands.csv")

merging the two files based on plate,mjd and fiberid

In [18]:
updated_file_1=pd.merge(bands,m_m,how="inner",on=["plate","mjd","fiberid"])

spectral lines file

In [19]:
l_red=pd.read_csv("speclines.csv")

In [20]:
l_red.drop("zwarning",axis=1,inplace=True)
l_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742749 entries, 0 to 742748
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   plate           742749 non-null  int64  
 1   mjd             742749 non-null  int64  
 2   fiberid         742749 non-null  int64  
 3   OII_3726_flux   742749 non-null  float64
 4   OII_3729_flux   742749 non-null  float64
 5   OIII_4959_flux  742749 non-null  float64
 6   OIII_5007_flux  742749 non-null  float64
 7   NII_6548_flux   742749 non-null  float64
 8   NII_6584_flux   742749 non-null  float64
 9   h_beta_flux     742749 non-null  float64
 10  h_alpha_flux    742749 non-null  float64
dtypes: float64(8), int64(3)
memory usage: 62.3 MB


# combining all data

In [21]:
updated_file_2=pd.merge(l_red,updated_file_1,how="inner",on=["plate","mjd","fiberid"])

getting total flux of OII,OIII,NII

In [22]:
updated_file_2['OII']=updated_file_2['OII_3726_flux']+updated_file_2["OII_3729_flux"]
updated_file_2['OIII']=updated_file_2['OIII_4959_flux']+updated_file_2["OIII_5007_flux"]
updated_file_2['NII']=updated_file_2['NII_6548_flux']+updated_file_2["NII_6584_flux"]

In [23]:
updated_file_2.drop(["OII_3726_flux","OII_3729_flux","OIII_4959_flux","OIII_5007_flux","NII_6548_flux","NII_6584_flux"],axis=1,inplace=True)

In [24]:
updated_file_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166714 entries, 0 to 166713
Data columns (total 30 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   plate          166714 non-null  int64  
 1   mjd            166714 non-null  int64  
 2   fiberid        166714 non-null  int64  
 3   h_beta_flux    166714 non-null  float64
 4   h_alpha_flux   166714 non-null  float64
 5   modelMag_u     166714 non-null  float64
 6   modelMag_g     166714 non-null  float64
 7   modelMag_r     166714 non-null  float64
 8   modelMag_i     166714 non-null  float64
 9   modelMag_z     166714 non-null  float64
 10  modelMagErr_r  166714 non-null  float64
 11  cModelMag_u    166714 non-null  float64
 12  cModelMag_g    166714 non-null  float64
 13  cModelMag_r    166714 non-null  float64
 14  cModelMag_i    166714 non-null  float64
 15  cModelMag_z    166714 non-null  float64
 16  subClass       163033 non-null  object 
 17  extinction_u   166714 non-nul

# marking AGN flags of subaclass as 1 (galaxies having active galactic nuclie)

using dummies

In [25]:
subclass=pd.get_dummies(updated_file_2["subClass"],drop_first=True)
subclass.head(5)

,AGN BROADLINE,BROADLINE,STARBURST,STARBURST BROADLINE,STARFORMING,STARFORMING BROADLINE
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,1,0,0,0
4,0,0,0,0,1,0


In [26]:
updated_file_2=pd.concat([updated_file_2,subclass["AGN BROADLINE"]],axis=1)

In [27]:
updated_file_2.drop("subClass",axis=1,inplace=True)

renaming models

In [28]:
updated_file_2.rename(columns = {'modelMag_u' : 'u', 'modelMag_g' : 'g','modelMag_r' : 'r','modelMag_i' : 'i','modelMag_z' : 'z','cModelMag_u' : 'c_u','cModelMag_g' : 'c_g','cModelMag_r' : 'c_r','cModelMag_i' : 'c_i','cModelMag_z' : 'c_z','LGM_TOT_P50' : 'mass','OH_P50':'metallicity','AGN BROADLINE':'AGN','h_beta_flux':'h_beta','h_alpha_flux':'h_alpha','modelMagErr_r':'err_r'}, inplace = True)

In [29]:
updated_file_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166714 entries, 0 to 166713
Data columns (total 30 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   plate         166714 non-null  int64  
 1   mjd           166714 non-null  int64  
 2   fiberid       166714 non-null  int64  
 3   h_beta        166714 non-null  float64
 4   h_alpha       166714 non-null  float64
 5   u             166714 non-null  float64
 6   g             166714 non-null  float64
 7   r             166714 non-null  float64
 8   i             166714 non-null  float64
 9   z             166714 non-null  float64
 10  err_r         166714 non-null  float64
 11  c_u           166714 non-null  float64
 12  c_g           166714 non-null  float64
 13  c_r           166714 non-null  float64
 14  c_i           166714 non-null  float64
 15  c_z           166714 non-null  float64
 16  extinction_u  166714 non-null  float64
 17  extinction_g  166714 non-null  float64
 18  exti

In [30]:
updated_file_2.head()

,plate,mjd,fiberid,h_beta,h_alpha,u,g,r,i,z,...,extinction_z,z_p,z_s,zwarning,metallicity,mass,OII,OIII,NII,AGN
0,266,51630,6,38.33226,127.3058,19.48419,18.32629,17.84381,17.52679,17.42098,...,0.066067,0.082959,0.052654,0,8.688159,9.363875,96.70390,48.104540,34.339915,0
1,266,51630,17,46.94843,206.2126,19.07068,17.61512,16.97374,16.57817,16.34779,...,0.092948,0.077703,0.064959,0,9.052521,10.138845,64.26882,20.426900,99.031750,0
2,266,51630,41,66.98206,351.3045,18.54270,17.05090,16.25521,15.79538,15.45037,...,0.066998,0.068409,0.065183,0,9.136789,10.678234,60.94239,27.251414,196.946670,0
3,266,51630,44,73.13292,332.5545,21.35108,20.03811,22.92755,20.53038,20.99983,...,0.064205,-9999.000000,0.154813,0,8.981265,10.483578,116.68561,36.836870,125.202030,0
4,266,51630,51,257.94090,1151.2110,17.37162,16.17903,15.50150,15.05806,14.78616,...,0.078609,0.070731,0.064902,0,9.050070,10.870276,313.23040,128.686930,750.267400,0


In [31]:
updated_file_2.to_csv("complete_rawdata.csv",index=False)